In [1]:
import nltk
import string
import re,os
from nltk.stem import PorterStemmer

In [2]:
stop_words=[]
ps = PorterStemmer()
with open('Stopword-List.txt','r') as f:
    stop_words = f.read().split()

print(stop_words)
for i,v in enumerate(stop_words):
    stop_words[i] = ps.stem(v)



['a', 'is', 'the', 'of', 'all', 'and', 'to', 'can', 'be', 'as', 'once', 'for', 'at', 'am', 'are', 'has', 'have', 'had', 'up', 'his', 'her', 'in', 'on', 'no', 'we', 'do']


In [3]:
re1=r"\w+(?:-\w+)?"
print(re.search(re1,"\"intelli-gence.\"").group())


intelli-gence


In [4]:
inverted_index = {}
docs=[]
count=0
re1=r'[a-z]+(?:-[a-z]+)?'
filenames=os.listdir('ResearchPapers/')
for i in filenames:
    doc_id=int(i.strip('.txt'))
    docs.append(doc_id)
    file = os.path.join('ResearchPapers',i)
    with open(file,'r',encoding='utf-8', errors='ignore') as f:
        text = f.read()
        text=text.lower().split()
        count+=len(text)
        for index, val in enumerate(text):
            match = re.search(re1,val)
            if match:
                val = match.group()
                if "-" in val:
                    val = val.replace("-","")
                val= ps.stem(val)
                if (val not in stop_words) and (len(val)>2):
                    if val in inverted_index:
                        if doc_id in inverted_index[val]:
                            inverted_index[val][doc_id].append(index)
                        else:
                            inverted_index[val][doc_id] = [index]
                    else:
                        inverted_index[val] = {doc_id: [index]}

sorted_inverted_index = dict(sorted(inverted_index.items()))
for key,nested_dic in sorted_inverted_index.items():
    sorted_inverted_index[key]=dict(sorted(nested_dic.items()))
# print(inverted_index)
docs = sorted(docs)

print("Total Tokkens: ",count)
print("Number of terms:",len(sorted_inverted_index))
print("Tokkens drops: ",(len(sorted_inverted_index)/count)*100)
print(sorted_inverted_index[ps.stem('cancer')])







Total Tokkens:  196879
Number of terms: 14728
Tokkens drops:  7.480736899313792
{2: [7318], 7: [30092, 34521], 8: [1783], 11: [132, 818, 828], 16: [7181, 13369], 22: [14286], 24: [745, 4210], 26: [611, 5211, 5286, 9658, 10358, 10389, 10391]}


In [5]:
def intersectTerms(term1,term2):
    inverted_index_1 = sorted_inverted_index[ps.stem(term1)]
    inverted_index_2 = sorted_inverted_index[ps.stem(term2)]
    doc_list=[]
    if len(inverted_index_1)<len(inverted_index_2):
        for i in inverted_index_1.keys():
            if i in inverted_index_2.keys():
                doc_list.append(i)
    else:
        for i in inverted_index_2.keys():
            if i in inverted_index_1.keys():
                doc_list.append(i)
    return doc_list
        
    

In [6]:
def intersectPTerms(term1,term2,k):
    inverted_index_1 = sorted_inverted_index[ps.stem(term1)]
    inverted_index_2 = sorted_inverted_index[ps.stem(term2)]
    doc_list=set()
    if len(inverted_index_1)<len(inverted_index_2):
        for i in inverted_index_1.keys():
            if i in inverted_index_2.keys():
                for x in inverted_index_1[i]:
                    for y in inverted_index_2[i]:
                        if abs(y-x)<=k:
                            doc_list.add(i)
    else:
        for i in inverted_index_2.keys():
            if i in inverted_index_1.keys():
                for x in inverted_index_2[i]:
                    for y in inverted_index_1[i]:
                        if abs(y-x)<=k:
                            doc_list.add(i)
    return sorted(list(doc_list))

In [7]:
def unionTerms(term1,term2):
    inverted_index_1 = set(sorted_inverted_index[ps.stem(term1)].keys())
    inverted_index_2 = set(sorted_inverted_index[ps.stem(term2)].keys())
    inverted_index_1.union(inverted_index_2)
    return sorted(list(inverted_index_1))
    

In [8]:
def inverseTerms(term):
    inverted_index_set = set(sorted_inverted_index[ps.stem(term)].keys())
    return list((set(docs)).difference(inverted_index_set))

In [9]:
print(sorted_inverted_index[ps.stem('transformer')].keys())
print(inverseTerms('transformer'))
print(docs)


dict_keys([2, 7, 8, 12, 16, 17, 18, 21, 22, 24, 26])
[1, 3, 9, 11, 13, 14, 15, 23, 25]
[1, 2, 3, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26]


In [38]:
user_input = (input("Enter query: ")).lower()
intersection_list=[]
inter_lists=[]
union_list=[]
inverse_list=[]
inv_list=[]
result = set()
if "and" in user_input: 
    inter_list = user_input.split(" and ") 
    for x in inter_list: 
        if "or" in x: 
            unions = x.split(" or ")
            for y in unions:
                if "not" in y:
                    inverse_term = y.lstrip("not ")
                    inv_list = inverseTerms(inverse_term)
                else:
                    inverse_term = y.strip()
                    inv_list = list(sorted_inverted_index[ps.stem(inverse_term)].keys())
                for i in inv_list:
                    union_list.append(i)
        else: 
            if "not" in x: 
                inverse_term = x.lstrip("not ") 
                inv_list = inverseTerms(inverse_term)
            else: 
                inverse_term = x.strip() 
                inv_list = list(sorted_inverted_index[ps.stem(inverse_term)].keys()) 
            for i in inv_list:
                union_list.append(i) 
        intersection_list.append(union_list) 
        union_list=[] 
    result=set(intersection_list[0]) 
    for x in intersection_list:
        result = result.intersection(set(x)) 
    # print(intersection_list)
    
elif "or" in user_input:
    unions = user_input.split(" or ")
    for y in unions:
        if "not" in y:
            inverse_term = y.lstrip("not ")
            inv_list = inverseTerms(inverse_term)
        else:
            inverse_term = y.strip()
            inv_list = list(sorted_inverted_index[ps.stem(inverse_term)].keys())
        for i in inv_list:
            union_list.append(i)
    result = set(union_list)

elif "not" in user_input:
    inverse_term = user_input.lstrip("not ")
    result = inverseTerms(inverse_term)
    
    
    
print(sorted(list(result)))
            
                    


[2, 7, 16, 22]


In [37]:
# NOT cancer AND NOT overview OR NOT heart
print(intersectTerms('overview','cancer'))
print(sorted_inverted_index[ps.stem('cancer')].keys())
print(sorted_inverted_index[ps.stem('overview')].keys())
# print(inverseTerms('overview'))
print(inverseTerms('cancer'))
# print(inverseTerms('heart'))
# print(sorted_inverted_index[ps.stem('heart')].keys())



[2, 7, 16, 22]
dict_keys([2, 7, 8, 11, 16, 22, 24, 26])
dict_keys([1, 2, 7, 12, 15, 16, 17, 22])
[1, 3, 9, 12, 13, 14, 15, 17, 18, 21, 23, 25]
